In [ ]:
import fitz  # pymupdf
import os
from PIL import Image
import sys
from mutagen.id3 import APIC, ID3, TIT2, TPE1, TCOM, TPE2, TRCK, USLT, TCON, TDRC, TALB

Lista de géneros:
      0.Blues
      1.Classic Rock
      2.Country
      3.Dance
      4.Disco
      5.Funk
      6.Grunge
      7.Hip-Hop
      8.Jazz
      9.Metal
     10.New Age
     11.Oldies
     12.Other
     13.Pop
     14.R&B
     15.Rap
     16.Reggae
     17.Rock
     18.Techno
     19.Industrial
     20.Alternative
     21.Ska
     22.Death Metal
     23.Pranks
     24.Soundtrack
     25.Euro-Techno
     26.Ambient
     27.Trip-Hop
     28.Vocal
     29.Jazz+Funk
     30.Fusion
     31.Trance
     32.Classical
     33.Instrumental
     34.Acid
     35.House
     36.Game
     37.Sound Clip
     38.Gospel
     39.Noise
     40.AlternRock
     41.Bass
     42.Soul
     43.Punk
     44.Space
     45.Meditative
     46.Instrumental Pop
     47.Instrumental Rock
     48.Ethnic
     49.Gothic
     50.Darkwave
     51.Techno-Industrial
     52.Electronic
     53.Pop-Folk
     54.Eurodance
     55.Dream
     56.Southern Rock
     57.Comedy
     58.Cult
     59.Gangsta
     60.Top 40
     61.Christian Rap
     62.Pop/Funk
     63.Jungle
     64.Native American
     65.Cabaret
     66.New Wave
     67.Psychedelic
     68.Rave
     69.Showtunes
     70.Trailer
     71.Lo-Fi
     72.Tribal
     73.Acid Punk
     74.Acid Jazz
     75.Polka
     76.Retro
     77.Musical
     78.Rock & Roll
     79.Hard Rock

In [ ]:
def tagMusic(archivo, tags):
    """
    Actualiza o agrega metadatos en un archivo MP3.
    
    Parámetros:
      archivo (str): Ruta del archivo MP3.
      tags (dict): Diccionario con claves entre:
         'title', 'artist', 'album', 'composer', 'album_artist',
         'genre', 'track', 'lyrics', 'year'.
         Ejemplo para 'track': "3/10"
    
    Se eliminan los frames existentes para evitar duplicados.
    Para la letra se utiliza USLT con encoding UTF-8, lang "XXX" y desc vacío.
    """
    try:
        audio = ID3(archivo)
    except Exception:
        audio = ID3()

    # Lista de tags que queremos actualizar
    frames = ["TIT2", "TPE1", "TALB", "TCOM", "TPE2", "TCON", "TRCK", "USLT", "TDRC"]
    for frame in frames:
        audio.delall(frame)

    if "title" in tags and tags["title"]:
        audio.add(TIT2(encoding=3, text=tags["title"]))
    if "artist" in tags and tags["artist"]:
        audio.add(TPE1(encoding=3, text=tags["artist"]))
    if "album" in tags and tags["album"]:
        audio.add(TALB(encoding=3, text=tags["album"]))
    if "composer" in tags and tags["composer"]:
        audio.add(TCOM(encoding=3, text=tags["composer"]))
    if "album_artist" in tags and tags["album_artist"]:
        audio.add(TPE2(encoding=3, text=tags["album_artist"]))
    if "genre" in tags and tags["genre"]:
        audio.add(TCON(encoding=3, text=tags["genre"]))
    if "track" in tags and tags["track"]:
        audio.add(TRCK(encoding=3, text=tags["track"]))
    if "lyrics" in tags and tags["lyrics"]:
        audio.add(USLT(encoding=3, lang="XXX", desc="", text=tags["lyrics"]))
    if "year" in tags and tags["year"]:
        audio.add(TDRC(encoding=3, text=str(tags["year"])))
        
    audio.save(archivo)

In [ ]:
def getTAGS(archivo):
	"""
	Devuelve un diccionario con los metadatos de un archivo MP3.
	Se intentan obtener: título, artista, álbum, compositor, artista del álbum,
	género, track (número/total), letra y año.
	
	Si un tag no está presente, su valor será None.
	"""
	try:
		audio = ID3(archivo)
	except Exception as e:
		return {}

	# Diccionario con las claves y los correspondientes frames ID3
	tags_frames = {
		"title": "TIT2",
		"artist": "TPE1",
		"album": "TALB",
		"composer": "TCOM",
		"album_artist": "TPE2",
		"genre": "TCON",
		"track": "TRCK",
		"lyrics": "USLT",
		"year": "TDRC"
	}
	tags = {}
	for key, frame_id in tags_frames.items():
		frame = audio.get(frame_id)
		if frame:
			try:
				# Los frames suelen tener un atributo .text que es una lista
				tags[key] = frame.text[0]
			except Exception:
				# En caso de cualquier otro tipo de dato, lo convertimos a cadena
				tags[key] = str(frame)
		else:
			tags[key] = None
	return tags

In [ ]:
# Ejemplo de uso:
nuevos_tags = {
    "title": "Mi Canción",
    "artist": "Mi Artista",
    "album": "Mi Álbum",
    "composer": "Mi Compositor",
    "album_artist": "Artista del Álbum",
    "genre": "Pop",
    "track": "1/10",
    "lyrics": "Aquí va la letra completa de la canción...",
    "year": "2021"
}
tagMusic("cancion.mp3", nuevos_tags)

Agregar directorio actual

A una lista de nombres les agrega el path absoluto de este documento.

In [ ]:
def addPath( lista ):
	path_actual = os.getcwd()											# Si se usa un .ipynb
	# path_actual = os.path.dirname(os.path.abspath(__file__))				# Si se usa un .py
	if isinstance(lista, list):  # Si es una lista, agregar la ruta a cada elemento
		return [os.path.join(path_actual, archivo) for archivo in lista]
	elif isinstance(lista, str):  # Si es una sola cadena, agregar la ruta
		return os.path.join(path_actual, lista)


UNIR PDF

In [ ]:
def unir_pdfs(entradas, salida, inicial_par=False , final_par=False):
	
	if not entradas:
		print("Error: La lista de PDFs está vacía.")
		return
	doc_final = fitz.open()

	for pdf in entradas:
		if os.path.exists(pdf):
			doc_temp = fitz.open(pdf)
			if inicial_par and len(doc_temp) % 2 != 0 :
				doc_final.new_page()
	
			doc_final.insert_pdf(doc_temp)

			if final_par and len(doc_temp) % 2 != 0 :
				doc_final.new_page()

			doc_temp.close()
		else:
			print(f"Advertencia: No se encontró '{pdf}' y será omitido.")

	if len(doc_final) > 0:
		doc_final.save(salida)
		print(f"PDF guardado como '{salida}'")
	else:
		print("Error: No se pudo generar el PDF de salida.")
		doc_final.close()
		return

	doc_final.close()

	print(f"PDFs combinados y guardados en '{salida}'")

In [ ]:
# Ejemplo de uso
archivos = ["archivo1.pdf", "archivo2.pdf", "archivo3.pdf"]
resultado = "resultado.pdf"

# Agrega el path del archivo actual
archivos = addPath(archivos)
resultado = addPath(resultado)

unir_pdfs(archivos, resultado)

SEPARAR PÁGINAS

In [ ]:
def separar_hojas(entrada, salida, paginas):
	"""Separa hojas específicas de un PDF y las guarda individualmente en la carpeta de salida.

	Parámetros:
	entrada (str): Ruta del archivo PDF de entrada.
	salida (str): Carpeta donde se guardarán los archivos.
	paginas (list): Lista de números de páginas a extraer (base 1).
	"""

	if not os.path.exists(entrada):
		print(f"Error: No se encontró '{entrada}'")
		return

	if not os.path.exists(salida):
		os.makedirs(salida)  # Crea la carpeta si no existe
		
	doc_inicial = fitz.open(entrada)
	nombre_base = os.path.splitext(os.path.basename(entrada))[0]  # Nombre sin extensión

	for num_pagina in paginas:
		if 1 <= num_pagina <= len(doc_inicial):
			doc_tmp = fitz.open()
			doc_tmp.insert_pdf(doc_inicial, from_page=num_pagina - 1, to_page=num_pagina - 1)
			archivo_salida = os.path.join(salida, f"{nombre_base}_pag_{num_pagina}.pdf")
			doc_tmp.save(archivo_salida)
			doc_tmp.close()
			print(f"Guardado: {archivo_salida}")
		else :
			print(f"Advertencia: La página {num_pagina} está fuera de rango y será omitida.")

	doc_inicial.close()

In [ ]:
# Ejemplo de uso
archivos = "documento.pdf"
resultado = "paginas_separadas"

# Agrega el path del archivo actual
archivos = addPath(archivos)
resultado = addPath(resultado)

separar_hojas(archivos , resultado , [1, 3, 5])

SEPARAR PDF's

In [ ]:
def separar_pdf(entrada, salida, paginas, concervar= False, salida_extra = None):
	if not os.path.exists(entrada):
		print(f"Error: No se encontró '{entrada}'")
		return

	doc_inicial = fitz.open(entrada)
	doc_salida = fitz.open()
	doc_salida_extra = fitz.open()

	for num_pagina in range(len(doc_inicial)):
		if num_pagina + 1 in paginas:
			doc_salida.insert_pdf(doc_inicial, from_page=num_pagina, to_page=num_pagina)
		elif concervar :
			doc_salida_extra.insert_pdf(doc_inicial, from_page=num_pagina, to_page=num_pagina)

	if len(doc_salida) > 0:
		doc_salida.save(salida)
		print(f"Guardado: {salida}")
	if len(doc_salida_extra) > 0:
		print(f"Guardado: {salida_extra}")

	doc_inicial.close()
	doc_salida.close()
	doc_salida_extra.close()

In [ ]:
# Ejemplo de uso
archivos = ["colores.pdf", "pagina2.pdf"]
resultado = "pagina1.pdf"

# Agrega el path del archivo actual
archivos = addPath(archivos)
resultado = addPath(resultado)

separar_pdf(archivos[0], resultado, [1], True, archivos[1])

WEBP a png

In [ ]:
def webp_to_png( webp_file ) :
	if webp_file.lower().endswith(".webp"):
		nueva_imagen = webp_file.replace(".webp", ".png")
		with Image.open(webp_file) as img:
			img.save(nueva_imagen, "PNG")
		return nueva_imagen
	return webp_file  # Si no es .webp, devolver la original

FOTO A PDF

In [ ]:
A4_WIDTH, A4_HEIGHT = 595, 842  


def imagenes_a_pdf(imagenes, salida, scale_widht = A4_WIDTH , scale_height = 842 ):
	"""Convierte una lista de imágenes en un archivo PDF.

	Parámetros:
	imagenes (list): Lista de rutas de imágenes a convertir.
	salida (str): Ruta del archivo PDF de salida.
	"""
	if not imagenes:
		print("Error: La lista de imágenes está vacía.")
		return

	doc = fitz.open()

	for imagen in imagenes:
		if os.path.exists(imagen):
			img_doc = fitz.open()  # Documento temporal
			img_doc.insert_page(0, width=scale_widht, height=scale_height)  # Página A4 en blanco
			rect = img_doc[0].rect  # Rectángulo de la página
			
			# Obtener el tamaño de la imagen
			img = fitz.Pixmap(imagen)
			img_width, img_height = img.width, img.height

			# Escalar la imagen manteniendo la proporción
			scale = min(A4_WIDTH / img_width, A4_HEIGHT / img_height)
			new_width = img_width * scale
			new_height = img_height * scale

			# Centrar la imagen en la página
			x0 = (A4_WIDTH - new_width) / 2
			y0 = (A4_HEIGHT - new_height) / 2
			x1 = x0 + new_width
			y1 = y0 + new_height
			img_rect = fitz.Rect(x0, y0, x1, y1)

			# Insertar la imagen escalada en la página
			img_doc[0].insert_image(img_rect, filename=imagen)
			
			# Agregar la página con la imagen al documento final
			doc.insert_pdf(img_doc)
			img_doc.close()
		else:
			print(f"Advertencia: No se encontró '{imagen}' y será omitida.")

	if len(doc) > 0:
		doc.save(salida)
		print(f"PDF guardado como '{salida}'")
	else:
		print("Error: No se pudo generar el PDF de salida.")

	doc.close()

In [ ]:
# Ejemplo de uso
archivos = ["rojo.jpg", "azul.png"]
resultado = "resultado.pdf"

# Agrega el path del archivo actual
archivos = addPath(archivos)
resultado = addPath(resultado)

imagenes_a_pdf(archivos, resultado)

Renombrar Masivo

In [ ]:
def renombrar(archivos , nombre , numerar = True, inicial = 1 , post_idx = False):
	"""Renombra archivos con un nombre base, opcionalmente numerándolos.

	Parámetros:
	archivos (list): Lista de archivos a renombrar (rutas completas o nombres).
	nombre (str): Nuevo nombre base sin extensión.
	numerar (bool): Si es True, agrega un número al nombre.
	inicial (int): Número inicial si numerar es True.
	post_idx (bool): Si es True, el número se coloca después del nombre y antes de la extensión.
					 Si es False, el número se coloca antes del nombre.
	"""
	for i, archivo in enumerate(archivos, start=inicial):
		if not os.path.exists(archivo):
			print(f"Advertencia: No se encontró '{archivo}', se omitirá.")
			continue
		
		ruta, ext = os.path.splitext(archivo)  # Obtener extensión del archivo
		dir_archivo = os.path.dirname(archivo)  # Directorio del archivo
		
		if numerar:
			if post_idx:
				nuevo_nombre = f"{nombre} #{i}{ext}"
			else:
				nuevo_nombre = f"{i}.{nombre}{ext}"
		else:
			nuevo_nombre = f"{nombre}{ext}"  # Si no se numera, solo usa el nombre base

		nueva_ruta = os.path.join(dir_archivo, nuevo_nombre)

		try:
			os.rename(archivo, nueva_ruta)
			print(f"Renombrado: '{archivo}' → '{nueva_ruta}'")
		except Exception as e:
			print(f"Error al renombrar '{archivo}': {e}")

In [ ]:
# Ejemplo de uso
archivos = ["documento1.pdf", "documento2.pdf", "imagen.jpg"]
resultado = "resultado.pdf"

# Agrega el path del archivo actual
archivos = addPath(archivos)
resultado = addPath(resultado)

renombrar(archivos, "archivo", numerar=True, inicial=10, post_idx=True)